# Transformer - Gerador e Resumo


### Descriçao

---
1. O objetivo deste notebook é aprender sobre Transformer
2. Vamos aprender sobre a biblioteca Transformer e construir um gerador de texto e também um resumo de paragrafos.
---

### Baixando os pacotes


In [ ]:
!pip install transformers

### Documentação

1. **Transformers**:
   - [Documentação do Hugging Face Transformers](https://huggingface.co/transformers/)
   - [Repositório GitHub](https://github.com/huggingface/transformers)


2. **Datasets**:
   - [Documentação do Hugging Face Datasets](https://huggingface.co/docs/datasets/)
   - [Repositório GitHub](https://github.com/huggingface/datasets)


### Importando as bibliotecas

In [ ]:
import transformers
from transformers import pipeline

### Gerador de Texto

In [ ]:
gerador = pipeline("text-generation", model="pierreguillou/gpt2-small-portuguese")
texto = "Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico."
resultado = gerador(texto, max_length=60, do_sample=True, temperature=0.7, top_k=50, top_p=0.85)
resultado[0]['generated_text']

### Preencher Lacunas

In [ ]:
mascarar = pipeline("fill-mask", model="neuralmind/bert-base-portuguese-cased")

In [ ]:
texto = mascarar("Existe uma chance do copo cair no [MASK]")
for x in range(len(texto)):
  print(texto[x])

### Perguntas e Respostas

In [ ]:
qea = pipeline("question-answering", model="pierreguillou/bert-base-cased-squad-v1.1-portuguese")

In [ ]:
texto = "Carl Sagan foi um cientista norte-americano. Sagan é autor de mais de 600 publicações científicas e também de mais de vinte livros de ciência e ficção científica."
pergunta = "Quantas publicações cientificas Carl Sagan tem publicado?"
resposta = qea(question=pergunta, context=texto)
print("Pergunta: ", pergunta)
print("Resposta: ", resposta['answer'])
print("Score: ", resposta['score'])

### Resumo

In [ ]:
resumidor = pipeline("summarization")
texto = """Tolkien criou um mundo, a Terra Média. Desenhou mapas que indicam a localização de cada rio, mata ou acidente geográfico importante. Estudioso e especialista do inglês antigo e medieval, davas aulas de lingüística e aproveitou seu conhecimento para incorporar uma série de elementos inéditos em seus livros. Assim criou um alfabeto e uma língua com regras de gramática e pronúncia. Em seguida imaginou um povo para habitar este mundo e deu a ele uma história, heróis para reverenciar e inimigos para temer. Criou um calendário, leis e uma mitologia. Reproduziu as árvores genealógicas de seus personagens principais com atenção aos detalhes de até seis gerações diferentes. Várias raças habitam esta região, incluindo seres humanos, Elfos, Trols, Duendes e, os mais famosos, os Hobbits. Todas estas informações estão em grande parte incorporadas no final do terceiro volume da trilogia, em seis apêndices somando quase 200 páginas."""
resumo = resumidor(texto, max_length=100, min_length=50)
resumo = resumo[0]['summary_text']
print(f"Resumo: {resumo}")